(ch:discrete-distribution)=
# 이산 확률 분포

**기본 설정**

Numpy와 Pandas 라이브러리를 각각 np와 pd로 불러온다.

In [1]:
import numpy as np
import pandas as pd

데이터프레임의 [chained indexing을 금지시키기 위한 설정](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy)을 지정한다.
Pandas 3.0 버전부터는 기본 옵션으로 지정된다.

In [2]:
pd.options.mode.copy_on_write = True

주피터 노트북에서 부동소수점의 출력을 소수점 이하 6자리로 제한한다.
아래 코드는 주피터 노트북에서만 사용하며 일반적인 파이썬 코드가 아니다.

In [3]:
%precision 6

'%.6f'

아래 코드는 데이터프레임 내에서 부동소수점의 출력을 소수점 이하 6자리로 제한한다.

In [4]:
pd.set_option('display.precision', 6)

데이터 시각화를 위해 `matplotlib.pyplot`를 `plt`라는 별칭으로 불러온다.

In [5]:
import matplotlib.pyplot as plt

**데이터 저장소 디렉토리**

코드에 사용되는 [데이터 저장소의 기본 디렉토리](https://github.com/codingalzi/DataSci/tree/master/data)를 지정한다.

In [6]:
data_url = 'https://raw.githubusercontent.com/codingalzi/DataSci/refs/heads/master/data/'

**주요 내용**

이산 확률 분포의 주요 개념을 소개한다.

- 이산 확률 변수
- 이산 확률 분포
- 확률 질량 함수
- 누적 분포 함수

## 이산 확률 변수

{numref}`%s장 <ch:probability>`에서 살펴본 확률 변수는 해당 변수에 할당될 수 있는 값과 그 값이
발생할 확률에 의해 정의되며, 확률 분포는 이를 정리한 결과다.

예를 들어 한 개의 정상적인 주사위를 던졌을 때 나올 수 있는 값을 가리키는 확률 변수를 $X$의
확률 분포는 다음과 같다.

| X | 1 | 2 | 3 | 4 | 5 | 6|
| :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| 확률 | 1/6 | 1/6 | 1/6 | 1/6 | 1/6 | 1/6 |

반면에 캘리포니아 주택가격 데이터셋에서 임의로 선택된 구역의 소득 범주를 가리키는 확류 변수 $X$의
확률 분포는 다음과 같다.

| X | 1 | 2 | 3 | 4 | 5 |
| :---: | :---: | :---: | :---: | :---: | :---: |
| 확률 | 0.041372 | 0.333011 | 0.361017 | 0.177992 | 0.086607 |

위 두 예제에서 확률 변수가 가리키는 값은 $x_1$, $x_2$, $x_3$ ... 등의 형식으로 하나, 둘, 셋 하며 셀 수 있다. 
이런 값을 가리키는 확률 변수를 **이산 확률 변수**<font size='2'>discrete random variable</font>,
이산 확률 변수가 따르는 확률 분포를 **이산 확률 분포**<font size='2'>discrete random distribution</font>라 한다.

:::{admonition} 단변량 분포
:class: note

하나의 확률 변수가 갖는 확률 분포를 **단변량 분포**<font size='2'>univariate distribution</font>라 한다.
:::

**예제: 소득 범주 이산 확률 분포**

캘리포니아 주택가격 데이터셋을 이용하여 앞서 언급된 소득 범주의 이산 확률 분포를 가리키는 데이터프레임을 직접 생성해본다.

캘리포니아 주택가격 데이터셋을 불러온 후에 가구중위소득과 주택중위가격만 활용한다.

In [7]:
housing = pd.read_csv(data_url+"california_housing.csv")
housing = housing.loc[:, ['median_income', 'median_house_value']]
housing

,median_income,median_house_value
0,8.3252,452600.0
1,8.3014,358500.0
2,7.2574,352100.0
3,5.6431,341300.0
4,3.8462,342200.0
...,...,...
20635,1.5603,78100.0
20636,2.5568,77100.0
20637,1.7000,92300.0
20638,1.8672,84700.0


중위주택가격이 50만을 초과하는 경우는 삭제하고 인덱스를 초기화한다.

In [8]:
house_value_max = housing['median_house_value'].max() # 500,001
mask = housing['median_house_value'] >= house_value_max
housing = housing[~mask]
housing = housing.reset_index(drop=True)
housing.index.name = 'district'
housing

,median_income,median_house_value
district,,
0,8.3252,452600.0
1,8.3014,358500.0
2,7.2574,352100.0
3,5.6431,341300.0
4,3.8462,342200.0
...,...,...
19670,1.5603,78100.0
19671,2.5568,77100.0
19672,1.7000,92300.0


중위소득 범주 특성을 추가한다.

In [9]:
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

housing

,median_income,median_house_value,income_cat
district,,,
0,8.3252,452600.0,5
1,8.3014,358500.0,5
2,7.2574,352100.0,5
3,5.6431,341300.0,4
4,3.8462,342200.0,3
...,...,...,...
19670,1.5603,78100.0,2
19671,2.5568,77100.0,2
19672,1.7000,92300.0,2


소득 범주별 그룹화하고 구간별 도수와 상대도수를 확인한다.

In [10]:
stratification = housing.groupby('income_cat', observed=True, group_keys=True)
stratified_count = stratification.count()
stratified_ratio = stratified_count/(housing.shape)[0]
stratified_ratio

,median_income,median_house_value
income_cat,,
1,0.041372,0.041372
2,0.333011,0.333011
3,0.361017,0.361017
4,0.177992,0.177992
5,0.086607,0.086607


위 데이터프레임을 이용하여 소득 범주에 대한 확률 분포를 구한다.

In [34]:
prob_income = stratified_ratio.iloc[:, 0:1]
prob_income.columns = ['확률']
prob_income.index.name = 'X'

prob_income = prob_income.T
prob_income

X,1,2,3,4,5
확률,0.041372,0.333011,0.361017,0.177992,0.086607


### 확률 질량 함수(PMF)

이산 확률 변수 $X$가 가리키는 값이 $x$이고 $x$가 발생할 확률이 $p$일 때 다음과 같이 표기한다.

$$
P(X=x) = p
$$

확률 변수 $X$가 $x$를 취할 수 있는 확률을 계산하는 
**확률 질량 함수**<font size='2'>probability mass function</font>(PMF)함수 $f$를 다음과 같이 정의한다.

$$
f(x) = P(X = x)
$$

**예제: 소득 범주의 PMF**

소득 범주의 PMF는 다음과 같이 정의할 수 있다.

In [12]:
def f_income(x):
    return (prob_income[x].values)[0]

예를 들어 임의의 구역의 소득 범주가 2일 확률은 약 33.3%다.

In [13]:
f_income(2)

0.333011

그런데 `f_income()` 함수는 사실상 `prob_income` 자체와 다르지 않다.
따라서 `prob_income` 자체를 PMF로 대신 사용한다.
단, 함수 호출 대신 데이터프레임 인덱싱 기능과 적절한 메서드를 사용한다.

**확률의 성질**

이산 확률 분포가 주어졌을 때 모든 경우에 대한 확률은 0과 1사이의 부동소수점이고, 
모든 확률의 합은 1이다.
즉 다음이 성립해야 한다.

- 첫째, 확률 변수가 취할 수 있는 임의의 값 $x_k$에 대한 확률은 0과 1 사이의 값이다.

    $$0 \le f(x_k) \le 1$$

- 둘째, 확률 변수가 취할 수 있는 모든 값에 대한 확률의 합은 1이다.

    $$\sum_k f(x_k) = 1$$


소득 범주의 확률 변수 또한 위 성질을 만족한다.

- 모든 확률은 0과 1사이의 값

In [14]:
(0 <= prob_income) & (prob_income <= 1)

X,1,2,3,4,5
확률,True,True,True,True,True


다음 방식도 가능하다.

In [15]:
((0 <= prob_income) & (prob_income <= 1)).all(axis=1)

확률    True
dtype: bool

- 모든 확률의 합은 1

In [16]:
prob_income.sum(axis=1)

확률    1.0
dtype: float64

### 누적 분포 함수(CDF)

확률 변수 $X$가 $x$보다 같거나 작은 값을 가질 확률을 계산하는 함수를
**누적 분포 함수**<font size='2'>cumulative distribution function</font>(CDF)라고 부르며
아래처럼 정의된다.

$$
F(x) = P(X \le x) = \sum_{x_k \le x} f(x_k)
$$

**소득 범주의 CDF**

소득 볌주의 CDF 또한 `prob_income`을 이용하여 정의된다.

In [118]:
def income_cdf(x):
    return np.sum([prob_income[x_k] for x_k in prob_income.columns if x_k <= x])

예를 들어 소득 범주가 1부터 3까지에 속할 확률은 약 73.5%다.

In [119]:
income_cdf(3)

0.735400

모든 소득 범주의 합이 1임을 아래 방식으로도 확인한다.

In [20]:
income_cdf(5)

1.000000

In [117]:
pts.max(axis=0)

array([94, 94])

## 다변량 분포

여러 개의 확률 변수들이 결합된 확률 분포를 **다변량 분포**<font size='2'>multivariate distribution</font>라 부른다.
여기서는 두 개의 이산 확률 변수가 결합된 다변량 분포의 기초 개념을 살펴 본다.

### 이산 결합 확률 분포

$X$, $Y$ 두 개의 이산 확률 변수가 주어졌을 때 `(X, Y)`는 각각의 확률 변수가 취할 수 있는 값들의 분포가 
**이산 결합 확률 분포**<font size='2'>discrete joint probability distribution</font>다.
이산 결합 확률 분포의 확률 질양 합수는 **결합 확률 질량 함수**<font size='2'>joint probability mass function</font>는
확률 변수 $X$는 $x$를, 확률 변수 $Y$는 $y$를 취할 확률을

$$
P(X=x, Y=y) = p
$$

로 표현한다.

## 연습문제

참고: [(연습) 이산 확률 분포](https://colab.research.google.com/github/codingalzi/DataSci/blob/master/practices/practice-discrete_distribution.ipynb)